In [2]:
#!/usr/bin/env python3

from cimaq_utils import *
from cimaq_utils import loadimages
from blind_rename import *
from inspect_misc_text import *
from inspect_misc_text import no_ascii
from zipctl import uzipfiles
from get_infos import get_infos
from fix_cimaq import get_cimaq_dir_paths
from get_zipinfos import get_zipinfos

from os.path import expanduser as xpu
from os.path import join as pjoin
from collections import OrderedDict
from tkinter import filedialog 
from zipfile import ZipFile
import codecs

from cimaq_filter import cimaq_filter
from fix_cimaq import get_cimaq_dir_paths
from zipctl import getnamelist
from zipctl import getinfolist
from chardet import UniversalDetector as Udet
# 'cimaq_derivatives/task_files/uzeprimes'

cimaq_dir="~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives"

In [2]:
print(79*'#')

###############################################################################


In [3]:
def get_cimaq_dir_paths(cimaq_dir="~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives"):
    json_params = json_read(xpu('~/cimaq_memory/cimaq_dir_list.json'), 'r')
    dlst = df.from_dict(json_params['dir_list'], orient='index', columns=['suffixes'])
    patterns =  df.from_dict(json_params['patterns'], orient='index', columns=['patterns'])
    dlst['fpaths'] = [join(xpu(cimaq_dir), sfx) for sfx in dlst.suffixes]
    return dlst.T, patterns


In [236]:
def get_zipinfos(fpath):
    with ZipFile(fpath, 'r') as myzip:
        encodings, bsheets, lterms = [], [], []
        namelist = getnamelist(fpath)
        contents = df(tuple(dict(zip(evenodd(itm)[0], evenodd(itm)[1])) for itm
                            in tuple(tuple(no_ascii(repr(itm)).strip().replace(
                       "'", "").replace("'",'').replace(
                           '=',' ')[:-2].split())[1:]
                       for itm in set(repr(myzip.getinfo(itm)).strip(' ').replace(
                                      itm, itm.replace(' ', '_')) 
                                      if ' ' in itm else repr(myzip.getinfo(itm)).strip(' ')
                                      for itm in namelist))))
        contents['fpaths'] = [join(os.path.splitext(fpath)[0], fname)
                              for fname in contents.filename]
        contents = contents.sort_values('filename')
        contents['src_names'] = namelist
        contents['exts'] = [os.path.splitext(filename)[1]
                            for filename in contents.filename]
        for row in contents.iterrows():
            '''https://www.prepressure.com/pdf/basics/fileformat#:~:text=PDF%20files%20are%20either%208,to%20create%20the%20PDF%20file).'''
            with myzip.open(row[1]['src_names']) as myfile:
                bsheets.append((row[1]['filename'], myfile.read()))
                myfile.seek(0)                   
                encod = chardet.detect((myfile.read()))['encoding']
                if encod == None:
                    encod = 'ascii'
                myfile.seek(0)                
                encodings.append((row[1]['filename'], encod))
                myfile.close()
        contents['encoding'] = [itm[1] for itm in sorted(encodings)]
        contents['bytes_sheets'] = [itm[1] for itm in sorted(bsheets)]
        contents['lineterminator'] = [[line for line in
                                       row[1]['bytes_sheets'].splitlines(
                                           keepends=True)][0].strip([line for line in
                                             row[1]['bytes_sheets'].splitlines()][0])
                                      for row in contents.iterrows()]
        contents['delimiter'] = [[csv.Sniffer().sniff(row[1]['bytes_sheets'].splitlines()[0].decode(row[1]['encoding'])).delimiter]
                                 for row in contents.iterrows()]
        contents['n_lines'] = [len(row[1]['bytes_sheets'].splitlines())
                               for row in contents.iterrows()]
        myzip.close()
    return contents.reset_index(drop=True)


In [237]:
contents_test = get_zipinfos(xpu('~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/task_files/zipped_eprime/3589314_893978_Tache_IRM.zip'))    
#.fpaths[8]
contents_test
#.iloc[0].filename

,filename,compress_type,filemode,external_attr,file_size,compress_size,fpaths,src_names,exts,encoding,bytes_sheets,lineterminator,delimiter,n_lines
0,3589314_893978_Tache_IRM/Debriefing_1.pdf,deflate,-rwxr-xr-x,0x4000,39105,25527,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Debriefing 1.pdf,.pdf,Windows-1254,b'%PDF-1.4\n1 0 obj\n<</Type /XObject /Subtype...,b'\n',[P],1122
1,3589314_893978_Tache_IRM/Debriefing_2.pdf,deflate,-rwxr-xr-x,0x4000,39649,32213,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Debriefing 2.pdf,.pdf,ascii,b'%PDF-1.4\n1 0 obj\n<</Type /XObject /Subtype...,b'\n',[P],869
2,3589314_893978_Tache_IRM/Encoding-scan-3589314...,deflate,-rwxr-xr-x,0x4000,94720,13589,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Encoding-scan-3589314...,.edat2,Windows-1254,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,b'\n',[\t],388
3,3589314_893978_Tache_IRM/Encoding-scan-3589314...,deflate,-rwxr-xr-x,0x4000,206476,12751,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Encoding-scan-3589314...,.txt,UTF-16,b'\xff\xfe*\x00*\x00*\x00 \x00H\x00e\x00a\x00d...,b'\r',[ ],9625
4,3589314_893978_Tache_IRM/Fiche_de_Suivi.pdf,deflate,-rwxr-xr-x,0x4000,28370,21577,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Fiche de Suivi.pdf,.pdf,ascii,b'%PDF-1.4\n1 0 obj\n<</Type /XObject /Subtype...,b'\n',[P],652
5,3589314_893978_Tache_IRM/Onset-Event-Encoding_...,deflate,-rwxr-xr-x,0x4000,21512,2826,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Onset-Event-Encoding_...,.txt,ascii,b'1 Enc Enc00 ...,b'\r\n',[ ],240
6,3589314_893978_Tache_IRM/Output-Responses-Enco...,deflate,-rwxr-xr-x,0x4000,3533,1190,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Output-Responses-Enco...,.txt,ascii,b'TrialNumber\tCategory\tTrialCode\tOldNumber\...,b'\r\n',[\t],121
7,3589314_893978_Tache_IRM/Output_Retrieval_CIMA...,deflate,-rwxr-xr-x,0x4000,5875,2213,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/Output_Retrieval_CIMA...,.txt,ISO-8859-1,b'category\tStim\tOldNumber\tRecognition_ACC\t...,b'\r\n',[\t],118
8,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.edat2,deflate,-rwxr-xr-x,0x4000,41472,5950,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.edat2,.edat2,Windows-1254,b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00...,b'\n',[\t],243
9,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.txt,deflate,-rwxr-xr-x,0x4000,67960,4472,/home/francois/../../media/francois/seagate_1t...,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.txt,.txt,UTF-16,b'\xff\xfe*\x00*\x00*\x00 \x00H\x00e\x00a\x00d...,b'\r',[ ],2695


In [188]:
contents_test.iloc[0]['fulnshrt']
#[['exts', 'encodings']]


array([b'\n', b'\r\n', b'\r'], dtype=object)

In [104]:
#### dlst, patterns = get_cimaq_dir_paths(cimaq_dir="~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives")
patterns, dlst

NameError: name 'patterns' is not defined

In [94]:



# zfiles['pscid'] = [next((itm for itm in fname.split('_')
#                          if len(itm) == 7 and itm.isnumeric()))
#                    for fname in zfiles.fname]
# zfiles['dccid'] = [next((itm for itm in fname.split('_')
#                          if len(itm) == 6 and itm.isnumeric()))
#                    for fname in zfiles.fname]


In [95]:
dlst, patterns = get_cimaq_dir_paths()
zfiles = loadfiles(loadimages(dlst.zeprimes.fpaths))
zfiles

,fname,ext,fpaths
0,3025432_658178_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
1,3123186_920577_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
2,3163875_199801_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
3,3291977_748676_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
4,3303819_893639_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
...,...,...,...
98,6258913_439776_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
99,6274286_336665_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
100,6281721_517070_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...
101,6334263_462345_Tache_IRM,.zip,/home/francois/../../media/francois/seagate_1t...


In [9]:
# allinfos = pd.concat([get_zipinfos(fpath) for fpath in zfiles.fpaths],
#                      axis=1, join='outer', ignore_index=True)
allinfos = [get_zipinfos(fpath) for fpath in zfiles.fpaths]


In [10]:
allinfos[10]

,filename,Information,compress_type,filemode,external_attr,file_size,compress_size,de,1.pdf,deflate,-rwxr-xr-x,0x4000,39105,2.pdf,39649,exts,archv_name,short_names,fpaths
0,3589314_893978_Tache_IRM/.DS_Store,NaN,deflate,-rw-r--r--,0x4000,6148,178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,3589314_893978_Tache_IRM,.DS_Store,/home/francois/../../media/francois/seagate_1t...
1,3589314_893978_Tache_IRM/Debriefing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,compress_type,filemode,external_attr,file_size,compress_size,NaN,NaN,,3589314_893978_Tache_IRM,Debriefing,/home/francois/../../media/francois/seagate_1t...
2,3589314_893978_Tache_IRM/Debriefing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,filemode,external_attr,file_size,NaN,compress_type,compress_size,,3589314_893978_Tache_IRM,Debriefing,/home/francois/../../media/francois/seagate_1t...
3,3589314_893978_Tache_IRM/Encoding-scan-3589314...,NaN,deflate,-rwxr-xr-x,0x4000,94720,13589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.edat2,3589314_893978_Tache_IRM,Encoding-scan-3589314-1,/home/francois/../../media/francois/seagate_1t...
4,3589314_893978_Tache_IRM/Encoding-scan-3589314...,NaN,deflate,-rwxr-xr-x,0x4000,206476,12751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.txt,3589314_893978_Tache_IRM,Encoding-scan-3589314-1,/home/francois/../../media/francois/seagate_1t...
5,3589314_893978_Tache_IRM/Fiche,NaN,deflate,-rwxr-xr-x,0x4000,28370,21577,Suivi.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,3589314_893978_Tache_IRM,Fiche,/home/francois/../../media/francois/seagate_1t...
6,3589314_893978_Tache_IRM/Onset-Event-Encoding_...,NaN,deflate,-rwxr-xr-x,0x4000,21512,2826,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.txt,3589314_893978_Tache_IRM,Onset-Event-Encoding_CIMAQ_3589314_session1A,/home/francois/../../media/francois/seagate_1t...
7,3589314_893978_Tache_IRM/Output-Responses-Enco...,NaN,deflate,-rwxr-xr-x,0x4000,3533,1190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.txt,3589314_893978_Tache_IRM,Output-Responses-Encoding_CIMAQ_3589314_session1A,/home/francois/../../media/francois/seagate_1t...
8,3589314_893978_Tache_IRM/Output_Retrieval_CIMA...,NaN,deflate,-rwxr-xr-x,0x4000,5875,2213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.txt,3589314_893978_Tache_IRM,Output_Retrieval_CIMAQ_3589314_893978_V03_1,/home/francois/../../media/francois/seagate_1t...
9,3589314_893978_Tache_IRM/PRATIQUE-3589314-1.edat2,NaN,deflate,-rwxr-xr-x,0x4000,41472,5950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.edat2,3589314_893978_Tache_IRM,PRATIQUE-3589314-1,/home/francois/../../media/francois/seagate_1t...


In [49]:
df(tuple(evenodd(tuple(range(0, 16)))))

,0,1,2,3,4,5,6,7
0,0,2,4,6,8,10,12,14
1,1,3,5,7,9,11,13,15


In [50]:
archv_path = zfiles.loc[0].fpaths
archv_path

'/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/task_files/zipped_eprime/3025432_658178_Tache_IRM.zip'

In [676]:

        contents[['pscid', 'dccid']] = \
                [(next((itm for itm in ar_name.split('_') if len(itm) \
                        == 7 and itm.isnumeric())),
                  next((itm for itm in ar_name.split('_')
                        if len(itm) == 6 and itm.isnumeric())))
                 for ar_name in contents.archv_name]
        for row in contents.iterrows():
            with myzip.open(row[1]['filename']) as myfile:
                rawsheets.append((row[1]['filename'], myfile.read()))
            myfile.close()
    myzip.close()
    contents = contents.sort_values('filename')
    contents['rawsheets'] = [rsheet[1] for rsheet in sorted(rawsheets)]
    contents['encodings'] = [chardet.detect(rsheet[1])['encoding']
                             for rsheet in sorted(rawsheets)]
    contents['hdr'] = [row[1]['rawsheets'].splitlines(keepends=False)[0][0] not
                       in b'.-0123456789' for row in contents.iterrows()]
    contents['decoded'] = [no_ascii(row[1]['rawsheets'].decode(row[1]['encodings']).lower())
                              if row[1]['encodings'] != None else
                              repr(row[1]['rawsheets']).lower()
                              for row in contents.iterrows()]
    
    contents['lineterminator'] = [row[1]['decoded'].splitlines(keepends=True)[0].strip(
                                     row[1]['decoded'].splitlines(keepends=False)[0])
                                  for row in contents.iterrows()]
    contents['delimiter'] = [csv.Sniffer().sniff(row[1]['decoded'].splitlines()[0]).delimiter
                             for row in contents.iterrows()]
    contents['repr_decoded'] = [str(row[1]['decoded']) for row in contents.iterrows()]
    
    contents['split_delim'] = [tuple(line.strip().replace(row[1]['delimiter'], '').replace(' ', '_').split(row[1]['delimiter'])+['\n']
#                                      if row[1]['delimiter']
                                     for line in row[1]['decoded'].splitlines(keepends=False))
                               for row in contents.iterrows()]    
    contents[['nrows', 'colsperrows']] = [(len(row[1]['rawsheets'].splitlines()),
                                                   tuple(len(line.split()) for line
                                                         in row[1]['decoded'].splitlines()))
                                          for row in contents.iterrows()]
    contents['dupindex'] = [evenodd(row[1]['decoded'].splitlines())[0][0] == \
                            evenodd(row[1]['decoded'].splitlines())[0][0]
                            for row in contents.iterrows()]
    
    contents['bytes_width'] = [len(line) for line in row[1]['rawsheets'].splitlines()
                               for line in row[1]['rawsheets'].splitlines()[1:]]
#     contents['dupindex'] = [evrows == odrows for (evrows, odrows) in
#                             evenodd(df(row[1]['split_delim']).index)
#                             for row in contents.iterrows()]    
#     contents['dupindex'] = [[(itm[0][0] == itm[1][0]) for itm in evenodd(row[1]['decoded'].splitlines())]
#                             for row in _.iterrows()]
    
#     contents['repr_noend_decoded'] = [line.join('\n') for line in repr(row[1]['decoded'].splitlines(keepends=False))
#                                       for row in contents.iterrows()]

    return contents.sort_values('filename').reset_index(drop=True)

In [677]:
all_bytes_sheets = get_zipinfos(archv_path)
all_bytes_sheets

/home/francois/.local/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


KeyError: 'rawsheets'

In [ ]:
# from decimal import Decimal
# from tabulate import tabulate

def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text
'_______________________'
delimsheets = all_bytes_sheets.loc[[2, 3, 4]]['split_delim']
decodsheets = all_bytes_sheets.loc[[2, 3, 4]]['decoded']
# flatten([line+'\n'.split() for line in decodsheets[2].splitlines()])
tuple(line.strip().split() for line in decodsheets[2].splitlines())
# evenodd(delimsheets[2])[0][0], evenodd(delimsheets[2])[1][0][0]
# tuple(str(round(float(itm), 1)) if itm.isnumeric() else itm for itm in delimsheets[4][44] )

In [ ]:
# df((itm for itm in delimsheets[2]))
# repr(json.dumps())
delimsheets[3]

In [ ]:
delimsheets[4]

In [ ]:
tabulate(delimsheets[3])
delimsheets[3]

In [668]:
def mksheet(inpt):
    evrows, odrows = evenodd(tuple(tuple(line.split()) for line in
                               inpt.splitlines()))
    return tuple(tuple(itm[0]+itm[1]) for itm in tuple(zip(evrows, odrows)))

testsheets = [mksheet(sheet) for sheet in 
# tuple('\t'.join("{:.2f}".format(float(itm)) if itm.isnumeric() else itm for itm in line)
#        for line in testsheets[2])
# dict(csv.Sniffer().sniff((testsheets[2]).splitlines()[0]))
# str.center(json.dumps(tuple(dict(testsheets[2]).items())))

SyntaxError: unexpected EOF while parsing (<ipython-input-668-26da57c16088>, line 10)

In [ ]:
help(float)

In [ ]:
testsheets[1] 

In [ ]:
testsheets[2]

In [ ]:


# display([no_ascii(line.strip(all_bytes_sheets.loc[2].delimiter)) for line in
#                     all_bytes_sheets.loc[2].repr_decoded.splitlines()])
# #         all_bytes_sheets.loc[3].repr_decoded,
#         all_bytes_sheets.loc[4].repr_decoded)

display(all_bytes_sheets.loc[6].repr_decoded)
#         all_bytes_sheets.loc[2].repr_noend_decoded
        #.rpartition(all_bytes_sheets.loc[2].delimiter),
        #.partition(all_bytes_sheets.loc[2].delimiter))
# display(all_bytes_sheets.iloc[8]['split_bytes_w_ends'][0],
# bytes(all_bytes_sheets.iloc[8]['split_bytes_w_ends'][0]))
#[-1]).decode('utf8')
# bytes(all_bytes_sheets.iloc[8]['split_bytes_n_ends'][0][-1])
#[['encodings', 'encodings2']]

In [ ]:
help(json.dumps)

In [ ]:
pd.Series([line.strip(bytes(itm for itm in line.strip()))
           for line in all_bytes_sheets.loc[3].rawsheets.splitlines()]).unique()[0]

In [ ]:
evesheet, oddsheet = evenodd(all_bytes_sheets.loc[0]['splitsheets'])
1#display(evesheet[0].split()
display(oddsheet[0].split(), oddsheet[0].split(all_bytes_sheets.loc[0]['delimiter']),
       '\t'.join(oddsheet[0].split(all_bytes_sheets.loc[0]['delimiter'])))

In [ ]:
all_bytes_sheets.loc[0]['splitlines_bytes'] = [line for line in all_bytes_sheets.loc[0]['rawsheets']]

In [ ]:
a_test = df(all_bytes_sheets.loc[0]['decodsheets'].split(all_bytes_sheets.loc[0]['lineterminator']))
b_test = df(all_bytes_sheets.loc[0]['decodsheets'].splitlines())

a_test, b_test

In [ ]:
pd.Series(all_bytes_sheets.loc[3]['splitsheets'][0].split(all_bytes_sheets.loc[3]['delimiter'])).unique()
#, all_bytes_sheets.loc[0]['splitsheets'], all_bytes_sheets.loc[0]['splitsheets']

In [ ]:
( all_bytes_sheets['fpaths'].values)#[0].replace('category', ' ').replace('stim', ' ')

In [456]:
tuple([itm[0].replace(subitm, '') for subitm in itm[1]] for itm in all_bytes_sheets.loc[0]['test_string'])

KeyError: 'test_string'

In [443]:
help(bytes(b'\xe7').title())

Help on bytes object:

class bytes(object)
 |  bytes(iterable_of_ints) -> bytes
 |  bytes(string, encoding[, errors]) -> bytes
 |  bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
 |  bytes(int) -> bytes object of size given by the parameter initialized with null bytes
 |  bytes() -> empty bytes object
 |  
 |  Construct an immutable array of bytes from:
 |    - an iterable yielding integers in range(256)
 |    - a text string encoded using the specified encoding
 |    - any object implementing the buffer API.
 |    - an integer
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(self, key, /)
 |      Return self[key].
 |  
 |  __getnewargs__(...)
 |  
 |  __g

In [444]:
len(all_bytes_sheets.loc[2]['splitsheets'])

1

In [479]:
all_bytes_sheets.iloc[4]['splitsheets']


['category\tstim\toldnumber\trecognition_acc\trecognition_resp\trecognition_rt\tspatial_resp\tspatial_rt\tspatial_acc(à corriger voir output-encodage)',
 'old\tanimal_penguin.bmp\told10\t1\t1\t3216\t8\t462',
 'old\tsporting_bicycle_old.bmp\told55\t0\t2\t9409\t8\t462',
 'old\tfruit_lemon_new.bmp\told69\t1\t1\t2746\t5\t4435',
 'new\tvegie_broccoli.bmp\tnew36\t1\t2\t1633\t5\t4435',
 'old\tanimal_zebra_old.bmp\told13\t1\t1\t1311\t8\t369',
 'old\tkitchen_apron.bmp\told27\t1\t1\t1105\t8\t379',
 'old\tanimal_monarch_old.bmp\told08\t1\t1\t1763\t5\t436',
 'new\tkitchen_toaster_old.bmp\tnew30\t1\t2\t1257\t5\t436',
 'new\tmusical_uculele_new.bmp\tnew39\t0\t1\t4342\t8\t455',
 'new\tfruit_cherry.bmp\told67\t1\t1\t1611\t6\t2122',
 'new\tfood_walnut01c.bmp\tnew34\t1\t2\t1348\t6\t2122',
 'old\tfood_pickle01a.bmp\told22\t1\t1\t1029\t8\t371',
 'new\tvegie_pumpkin.bmp\tnew21\t1\t2\t1795\t8\t371',
 'old\tanimal_lion_new.bmp\told07\t1\t1\t12322\t8\t488',
 'new\tsporting_flippers.bmp\tnew02\t1\t2\t1267\t8\t

In [336]:
[no_ascii(line).encode().strip().replace(b'\'', b'\s').strip().split() for line in all_bytes_sheets['decodsheets'].loc[7].split()]

[[b'\\sb\\\\s%PDF-1.4\\\\n%\\\\xf8\\\\xe5\\\\xd0\\\\xc4\\\\xc6\\\\n1'],
 [b'0'],
 [b'obj\\\\n<<'],
 [b'/Type'],
 [b'/Catalog\\\\n/Pages'],
 [b'2'],
 [b'0'],
 [b'R\\\\n>>\\\\nendobj\\\\n\\\\n2'],
 [b'0'],
 [b'obj\\\\n<<'],
 [b'/Type'],
 [b'/Pages\\\\n/Kids'],
 [b'[4'],
 [b'0'],
 [b'R'],
 [b'16'],
 [b'0'],
 [b'R'],
 [b'23'],
 [b'0'],
 [b'R'],
 [b'42'],
 [b'0'],
 [b'R'],
 [b'54'],
 [b'0'],
 [b'R'],
 [b'79'],
 [b'0'],
 [b'R]\\\\n/Count'],
 [b'6\\\\n>>\\\\nendobj\\\\n\\\\n3'],
 [b'0'],
 [b'obj\\\\n<<'],
 [b'/ProcSet'],
 [b'[/PDF'],
 [b'/Text'],
 [b'/ImageB'],
 [b'/ImageC'],
 [b'/ImageI]\\\\n/XObject'],
 [b'<<'],
 [b'/XIPLAYER0'],
 [b'6'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM1'],
 [b'8'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM2'],
 [b'10'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM3'],
 [b'11'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM4'],
 [b'12'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM5'],
 [b'13'],
 [b'0'],
 [b'R\\\\n/XIPLAYER_CM6'],
 [b'14'],
 [b'0'],
 [b'R\\\\n>>\\\\n\\\\n>>\\\\nendobj\\\\n\\\\n4'],
 [b'0'],
 [b'obj\\\\n<<'

In [203]:
all_bytes_sheets['decodsheets'].loc[5]

'TrialNumber\tCategory\tTrialCode\tOldNumber\tCorrectSource\tStim_RESP\tStim_RT\tStim_ACC\r\n1\tCTL\tCTL0\t\t5\t1\t1183\t0\r\n2\tEnc\tEnc00\t\t8\t1\t2759\t0\r\n3\tEnc\tEnc000\t\t9\t1\t1656\t0\r\n4\tCTL\tCTL01\t\t8\t1\t1402\t0\r\n5\tEnc\tEnc01\tOld60\t6\t\t0\t1\r\n6\tEnc\tEnc02\tOld01\t5\t1\t1822\t0\r\n7\tEnc\tEnc03\tOld24\t6\t1\t1930\t0\r\n8\tEnc\tEnc04\tOld68\t9\t1\t1814\t0\r\n9\tEnc\tEnc05\tOld55\t8\t1\t1094\t0\r\n10\tEnc\tEnc06\tOld46\t9\t1\t2895\t0\r\n11\tCTL\tCTL02\t\t5\t1\t838\t0\r\n12\tCTL\tCTL03\t\t6\t1\t807\t0\r\n13\tEnc\tEnc07\tOld38\t8\t1\t2139\t0\r\n14\tCTL\tCTL04\t\t8\t1\t791\t0\r\n15\tEnc\tEnc08\tOld47\t6\t1\t1678\t0\r\n16\tEnc\tEnc09\tOld20\t5\t1\t2383\t0\r\n17\tCTL\tCTL05\t\t8\t1\t840\t0\r\n18\tEnc\tEnc10\tOld30\t6\t1\t2578\t0\r\n19\tEnc\tEnc11\tOld05\t6\t1\t1558\t0\r\n20\tEnc\tEnc12\tOld25\t8\t1\t2848\t0\r\n21\tEnc\tEnc13\tOld77\t5\t1\t2340\t0\r\n22\tEnc\tEnc14\tOld17\t9\t1\t2740\t0\r\n23\tEnc\tEnc15\tOld35\t9\t1\t2095\t0\r\n24\tEnc\tEnc16\tOld32\t5\t1\t1966\t0\r\n25\t

In [119]:
display(chardet.detect(all_bytes_sheets.iloc[3]['rawsheets']))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}

In [225]:
dlst, patterns = get_cimaq_dir_paths()
uzipfiles(indir = dlst.zeprimes.fpaths,
          outdir = dlst.uzeprimes.fpaths)
cimaq_filter()
prefixes = pd.Series(('_'.join(val.split('_')[:2])
                      for val in loadfiles(loadimages(dlst.zeprimes.fpaths)).fname)).unique()
indiv_patterns = tuple(item for item
                           in dict(zip(prefixes, prefixes)).items())
prefixes, indiv_patterns
#     {"indiv_patterns": tuple(item for item\\
#                        in dict(zip(prefixes, prefixes)).items()),
#      "prefx": pd.Series(('_'.join(val.split('_')[:2]) for val in
#                          loadfiles(loadimages(dlst['uzeprimes'][0])).fname)).unique()}

KeyError: 'cimaq_dir'

(array([], dtype=float64), ())

In [2]:

def fix_cimaq(cimaq_dir):
    # Paths to files and direcotries
    cimaq_dir = xpu(cimaq_dir)
    dlst, prefixes = get_cimaq_dir_paths(cimaq_dir)

    # Name and indexing patterns and prefixes
#     mean_sheets_patterns = (('dccid', '(?<!\d)\d{6}(?!\d)'),
#                             ('pscid', '(?<!\d)\d{7}(?!\d)'))
    prefixes = pd.Series(('_'.join(val.split('_')[:2])
                          for val in loadfiles(dlst['uzeprimes'][0]).fname)).unique()
    indiv_patterns = tuple(item for item
                               in dict(zip(prefixes, prefixes)).items())
    
    # Structure file paths and participant identifiers in a DataFrame
    events_sheets = loadfiles(indir=dlst['uzeprimes'][0])
    confounds_sheets = loadfiles(dlst['confdir'][0])
    events_sheets['pscid'] = [re.compile('(?<!\d)\d{7}(?!\d)').search(fname).group()
                       for fname in events_sheets.fname]
    
    confounds_sheets['dccid'] = [re.compile('(?<!\d)\d{6}(?!\d)').search(fname).group()
                       for fname in loadfiles(dlst['confdir'][0]).fname]
    
    # Get parsing, dialect and encoding information with 'get_infos' function
    events_infos = df((get_infos(fpath) for fpath in events_sheets.fpaths))
    confounds_infos = df((get_infos(fpath) for fpath in confounds_sheets.fpaths))
    events_infos = sortmap(pd.merge(events_infos, events_sheets,
                                    how='outer'), indiv_patterns)
    confounds_infos = pd.merge(confounds_infos, confounds_sheets, how='outer')
    mean_parsing_infos = df((get_infos(fpath) for fpath in
                             loadfiles(pathlist = dlst['mean_paths']).fpaths))
    
    # Group information about participants who passed QC assessment
    new_mean_sheets = OrderedDict((row[1]['fpaths'],
                                   prep_sheet(filename = row[1]['fpaths'],
                                encoding=row[1]['encoding'],
                                hdr = row[1]['has_header'],
                                n_fields = row[1]['n_fields'],
                                delimiter = row[1]['delimiter'],
                                dupindex = row[1]['dup_index'],
                                row_breaks = row[1]['row_breaks'],
                                r_rowbreaks = row[1]['r_rowbreaks'],
                                n_lines = row[1]['n_lines'],
                                width = row[1]['width'],
                                lineterminator = row[1]['lineterminator']))
                     for row in mean_parsing_infos.iterrows())
    new_mean_sheets = dict(zip(new_mean_sheets.keys(),
                               tuple(sheet.convert_dtypes(str)
                                     for sheet in rename_imposter_cols(
                   [sheet for sheet in new_mean_sheets.values()],
                   mean_sheets_patterns))))
    min_ind = df([sheet.dccid for sheet in new_mean_sheets.values()
               if sheet.shape[0] == pd.Series(
                   [sheet.shape[0] for sheet in
                    new_mean_sheets.values()]).min()]).T
    inds = [sheet[1].dccid.tolist() for sheet in new_mean_sheets.items()]
    new_mean_sheets = dict((item[0], item[1].set_index(
                          'dccid').loc[min_ind.dccid].reset_index(drop=False))
                           for item in new_mean_sheets.items())
    renamer = [sheet[['dccid', 'pscid']] for sheet in new_mean_sheets.values()
               if 'dccid' and 'pscid' in sheet.columns][0].astype(str).dropna()
    events_infos = pd.merge(events_infos, renamer, on='pscid', how='outer').dropna(how='any')
    confounds_infos = pd.merge(confounds_infos, renamer, on='dccid', how='outer').dropna(how='any')
    all_infos = pd.concat([events_infos, confounds_infos]).fillna(False).reset_index(drop=True)
    all_infos['confounds'] = ['confounds' in fname for fname in all_infos.fname]
    all_infos['prefix'] = [find_key(row[1][['output_responses', 'onset_event',
                               'output_retrieval', 'confounds']].T.to_dict(), True)
                           for row in all_infos.iterrows()]
    all_infos.to_csv(join(cimaq_dir, 'all_parsing_infos.csv'), index=False)

    # Use 'prep_sheet' function to fix errors by reading data as a stream of bytes
    new_events = tuple((''.join(['sub-', row[1]['pscid'], '-', row[1]['dccid'],
                                '_', row[1]['prefix'], '.csv']),
                        prep_sheet(filename = row[1]['fpaths'],
                                   encoding=row[1]['encoding'],
                                   hdr = row[1]['has_header'],
                                   n_fields = row[1]['n_fields'],
                                   delimiter = row[1]['delimiter'],
                                   dupindex = row[1]['dup_index'],
                                   row_breaks = row[1]['row_breaks'],
                                   r_rowbreaks = row[1]['r_rowbreaks'],
                                   n_lines = row[1]['n_lines'],
                                   width = row[1]['width'],
                                   lineterminator = row[1]['lineterminator']))
                       for row in tqdm(all_infos.iterrows()))
    newdir = join(cimaq_dir, 'cleaned_events_and_confounds')
    os.makedirs(newdir, exist_ok=True)
    next((itm[1].to_csv(join(newdir, itm[0]), index=False)
          for itm in new_events))
    
def main():
    fix_cimaq(cimaq_dir = '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives')

if __name__ == "__main__":
    main()


373it [01:38,  3.78it/s]


In [194]:
# fp = xpu('~/cimaq_memory/cimaq_dir_list.json')
cimaq_dir = xpu('~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives')


{'mean_paths': ['/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/MotionResults/fMRI_meanMotion.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/MemoTaskParticipantFile.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/Neuropsych/ALL_Neuropsych_scores.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/Participants_bids.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/sub_list_TaskQC.tsv',
  '/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/participants/TaskResults/fMRI_behavMemoScores.tsv'],
 'new_mean_dir': ['/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/mean_sheets'],
 'confdir': ['/home/francois/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives/confounds'],
 'uzeprimes': 

In [190]:
read2 = json_read(fp2, 'rb')
read2

{'confdir': ['confounds'],
 'mean_paths': ['participants/MotionResults/fMRI_meanMotion.tsv',
  'participants/MemoTaskParticipantFile.tsv',
  'participants/Neuropsych/ALL_Neuropsych_scores.tsv',
  'participants/Participants_bids.tsv',
  'participants/sub_list_TaskQC.tsv',
  'participants/TaskResults/fMRI_behavMemoScores.tsv'],
 'new_mean_dir': ['mean_sheets'],
 'uzeprimes': ['task_files/uzeprimes']}

In [37]:
def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
    """
    Context manager to set the timestamp of the path to plus or
    minus a fixed delta, regardless of modifications within the context.

    if set_new is True, the delta is added. Otherwise, the delta is subtracted.
    """
    stats = os.stat(path)
    if set_new:
        new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
    else:
        new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
    try:
        yield
    finally:
        os.utime(path, ns=new_timestamp)


# Public Methods 

In [30]:
fp = xpu('~/cimaq_memory/cimaq_dir_list.json')

os.stat(fp)

# def json_read(fp):
#     with open(join(dname(fp), bname(fp)), 'r') as outfile:
#         json.load(outfile, cls=None, object_hook=None, parse_float=None,
#                   parse_int=None, parse_constant=None, object_pairs_hook=None)
#         outfile.close()
# fp = xpu('~/cimaq_memory/cimaq_dir_list.json')
# json.load(fp, cls=None, object_hook=None, parse_float=None,
#           parse_int=None, parse_constant=None, object_pairs_hook=None)
        
# json_read(fp)        

os.stat_result(st_mode=33204, st_ino=46792720, st_dev=2049, st_nlink=1, st_uid=1000, st_gid=1000, st_size=377, st_atime=1613319019, st_mtime=1613319019, st_ctime=1613319019)